In [3]:
import pandas as pd
import numpy as np
from utils.load import load_data_methodology_random_cutting
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from utils.metrics import f1
from utils.tuner import CVTuner
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import joblib
import keras_tuner
import tensorflow as tf

In [4]:
df = pd.read_csv("../Metadata/data_curated.csv")
df.set_index("File", inplace=True)
df.head()

,Class
File,
S1V3C1M3A Clip166.npy,1
S1V1C1M1A Clip18.npy,1
S1V4C1M4A Clip241.npy,1
S1V4C1M4A Clip239.npy,1
S1V4C1M4A Clip245.npy,1


In [5]:
X_train, y_train, X_val, y_val = load_data_methodology_random_cutting(df,["M1"],30)

In [6]:
X_train.shape

(2250, 30, 1662)

In [7]:
y_train.shape

(2250, 6)

In [8]:
X_val.shape

(750, 30, 1662)

In [9]:
y_val.shape

(750, 6)

# Cross Validation

In [10]:
cv = MultilabelStratifiedKFold(n_splits=5)
joblib.dump(cv, './cv/cv.joblib')

['./cv/cv.joblib']

# Hyper Parameter Tunning Recurrent Neural Networks

In [11]:
def build_model_LSTM(hp):
    
    model = Sequential()


    model.add(LSTM(
        hp.Int('input_units',min_value=10,max_value=70,step=10),
        return_sequences=True,
        activation='relu', 
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal(),
        recurrent_dropout=0.1,
        input_shape=(30,1662)))
    
    model.add(LSTM(
        hp.Int('GRU_2',min_value=10,max_value=70,step=10), 
        recurrent_dropout=0.1,
        return_sequences=False,
        activation='relu',
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal()
        ))
    

    model.add(Dense(
        hp.Int(f"Dense_units_1",min_value=10,max_value=70,step=10),
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
        ))
    
    model.add(Dense(6, activation='softmax'))

    opt = tf.keras.optimizers.Nadam(clipnorm = 1.0)
    model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=[f1])

    return model

In [12]:
tuner_LSTM = CVTuner(
                    data_cv= cv,
                    goal = 'f1',
                    hypermodel=build_model_LSTM,
                    oracle=keras_tuner.oracles.BayesianOptimizationOracle(
                    objective=keras_tuner.Objective('f1',direction = "max"),
                        max_trials=5                  
                    ),
                    directory='./experiments/',
                    project_name = 'LSTM',
                    overwrite=True
                    )

In [13]:
tuner_LSTM.search(X_train, y_train, 64, epochs=50)

Trial 5 Complete [00h 13m 06s]
f1: 0.7382549047470093

Best f1 So Far: 0.9621736407279968
Total elapsed time: 01h 08m 50s


In [14]:
df_LSTM = pd.DataFrame(tuner_LSTM.trial_scores)

In [15]:
df_LSTM

,trial_id,hyperparameters,f1,f1_std,precision,precision_std,recall,recall_std
0,0,"{'input_units': 40, 'GRU_2': 60, 'Dense_units_...",0.966592,0.000000,0.968750,0.000000,0.964444,0.000000
1,0,"{'input_units': 40, 'GRU_2': 60, 'Dense_units_...",0.955555,0.005518,0.962153,0.006597,0.960000,0.004444
2,0,"{'input_units': 40, 'GRU_2': 60, 'Dense_units_...",0.972191,0.006911,0.965860,0.007517,0.963704,0.006372
3,0,"{'input_units': 40, 'GRU_2': 60, 'Dense_units_...",0.956618,0.006951,0.963816,0.007410,0.961667,0.006550
4,0,"{'input_units': 40, 'GRU_2': 60, 'Dense_units_...",0.959911,0.006319,0.963463,0.006665,0.960889,0.006061
5,1,"{'input_units': 50, 'GRU_2': 40, 'Dense_units_...",0.875700,0.000000,0.882618,0.000000,0.868889,0.000000
6,1,"{'input_units': 50, 'GRU_2': 40, 'Dense_units_...",0.947720,0.036010,0.915697,0.033078,0.907778,0.038889
7,1,"{'input_units': 50, 'GRU_2': 40, 'Dense_units_...",0.955555,0.035940,0.928983,0.032901,0.923704,0.038929
8,1,"{'input_units': 50, 'GRU_2': 40, 'Dense_units_...",0.973333,0.037190,0.940071,0.034361,0.936111,0.039981
9,1,"{'input_units': 50, 'GRU_2': 40, 'Dense_units_...",0.924276,0.033719,0.937324,0.031220,0.933333,0.036189


In [16]:
df_LSTM.to_csv("./experiments/LSTM.csv")

In [17]:
df_LSTM.to_excel("./experiments/LSTM.xlsx")

In [ ]:
#disp = confusion_matrix_plot(y_val, y_pred)
#disp.plot()